# Advent of Code - Day 20

In [1]:
import re
import functools as ft
import itertools as it
import operator

import collections
import math

import copy

## Part 1

In [2]:
def make_tile(tile_string_in):
    return [list(nl) for nl in tile_string_in.splitlines()]

In [3]:
make_tile('''..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###''')

[['.', '.', '#', '#', '.', '#', '.', '.', '#', '.'],
 ['#', '#', '.', '.', '#', '.', '.', '.', '.', '.'],
 ['#', '.', '.', '.', '#', '#', '.', '.', '#', '.'],
 ['#', '#', '#', '#', '.', '#', '.', '.', '.', '#'],
 ['#', '#', '.', '#', '#', '.', '#', '#', '#', '.'],
 ['#', '#', '.', '.', '.', '#', '.', '#', '#', '#'],
 ['.', '#', '.', '#', '.', '#', '.', '.', '#', '#'],
 ['.', '.', '#', '.', '.', '.', '.', '#', '.', '.'],
 ['#', '#', '#', '.', '.', '.', '#', '.', '#', '.'],
 ['.', '.', '#', '#', '#', '.', '.', '#', '#', '#']]

In [4]:
def match_up(t1, t2):
    '''True if t2 can be placed above t1'''
    return t1[0]==t2[-1]

In [5]:
def match_down(t1, t2):
    '''True if t2 can be placed below t1'''
    return t1[-1]==t2[0]

In [6]:
def match_left(t1, t2):
    '''True if t2 can be placed to the left of t1'''
    return [r[0] for r in t1]==[r[-1] for r in t2]

In [7]:
def match_right(t1, t2):
    '''True if t2 can be placed to the right of t1'''
    return [r[-1] for r in t1]==[r[0] for r in t2]

In [8]:
def rotate_right(tile_in):
    
    return [[r[i] for r in tile_in][::-1]  # hacky non in-place reverse
            for i in range(len(tile_in[0]))]

In [9]:
def flip(tile_in):
    
    return [r[::-1] for r in tile_in]

OK, that should be enough basic functions for the moment.

Now, let's see whether we can get a cross of four tiles. I don't know how long that will take to solve...

In [10]:
def parse_input(str_in):
    
    return {int(tile_num): make_tile(tile_str.strip())
            for (tile_num, tile_str) in 
                re.findall('Tile (\d+):(\W+)', str_in, flags=re.MULTILINE)}

Let's see if we can find the corners. Start by seeing how many shared edges there are:

Actually, the problem states that "the outermost edges won't line up with any other tiles", so it's a lot easier than it looks at first. Of course, that's just for part 1...

We're only after the edges that don't appear anywhere else. Remember that they can appear in both directions, so as a hack, I'm going to include them in each direction:

In [11]:
def get_top_edge(grid_in):
    return grid_in[0]

In [12]:
def get_bottom_edge(grid_in):
    return grid_in[-1]

In [13]:
def get_left_edge(grid_in):
    return [r[0] for r in grid_in]

In [14]:
def get_right_edge(grid_in):
    return [r[-1] for r in grid_in]

In [15]:
def find_edges(tile_in):
    
    return [tile_in[0],
           tile_in[0][::-1],
           tile_in[-1],
           tile_in[-1][::-1],
           rotate_right(tile_in)[0],
           rotate_right(tile_in)[0][::-1],
           rotate_right(tile_in)[-1],
           rotate_right(tile_in)[-1][::-1]]
            
            

In [16]:
def find_edge_counts(tiles_in):
    '''Return the number of unique edges, and their counts.
       Need to convert to tuples as lists aren't hashable.'''
    edge_counts=[tuple(x) for y in 
                 [find_edges(tile) for tile in tiles_in.values()]
                 for x in y]

    return collections.Counter(edge_counts)

In [17]:
def find_exposed_edges(tiles_in):
    '''Find the number of exposed edges per tile'''

    exposed_edges_ls=[edge for (edge, count) 
                      in find_edge_counts(tiles_in).items()
                      if count==1]
    
    tile_exposed_edges_dict={}
    
    for (tile, grid) in tiles_in.items():
        tile_exposed_edges_dict[tile]=[edge for edge in find_edges(grid)
                                       if tuple(edge) in exposed_edges_ls]
                             
    
    return {tile:exposed_edges
            for (tile, exposed_edges) in tile_exposed_edges_dict.items()}

In [18]:
def count_exposed_edges(tiles_in):
    '''Find the number of exposed edges per tile'''

    exposed_edges_ls=[edge for (edge, count) 
                      in find_edge_counts(tiles_in).items()
                      if count==1]
    
    tile_exposed_edges_dict={}
    
    for (tile, grid) in tiles_in.items():
        tile_exposed_edges_dict[tile]=[edge for edge in find_edges(grid)
                                       if tuple(edge) in exposed_edges_ls]
                             
    
    return {tile:len(exposed_edges)//2 
            for (tile, exposed_edges) in tile_exposed_edges_dict.items()}

In [19]:
def day20_part1(file_in):
    
    tiles_dict=parse_input(open(file_in).read())
    
    return ft.reduce(operator.mul, 
                     [t for (t, c) in count_exposed_edges(tiles_dict).items()
                      if c==2])

In [20]:
assert day20_part1('data/day20_test')==20899048083289

In [21]:
day20_part1('data/day20_input')

20033377297069

## Part 2

Hmmm, some work's going to be needed here. First we need to complete the grid.

Start by choosing a corner piece:

Start by putting one of the corners in, in the correct orientation. So we need the single edges facing up and to the left.

In [22]:
def get_first_tile(tiles_in):
    '''return the id of the tile, and the grid
       in appropriate orientation for NW corner'''
    
    (corner_tile, edges)=[(tile, es) for (tile, es) in find_exposed_edges(tiles_in).items()
                          if len(es)==4][0]
    
    tile_grid=tiles_in[corner_tile]
    
    while (get_top_edge(tile_grid) not in edges) \
                or (get_left_edge(tile_grid) not in edges):

        tile_grid=rotate_right(tile_grid)
    

    return (corner_tile, tile_grid)

In [23]:
def placable_oriented_tile_on_grid(grid, tile, x, y):
    '''Place tile on grid at x, y in whatever orientation is 
       necessary. Return True if tile can be placed, False 
       otherwise. Does not place the tile'''
    if x>0 and grid[y][x-1]:
        if get_right_edge(grid[y][x-1]) != get_left_edge(tile):
            return False
    if y>0 and grid[y-1][x]:
        if get_bottom_edge(grid[y-1][x]) != get_top_edge(tile):
            return False
    if x<len(grid)-1 and grid[y][x+1]:
        if get_left_edge(grid[y][x+1]) != get_right_edge(tile):
            return False
    if y<len(grid)-1 and grid[y+1][x]:
        if get_top_edge(grid[y+1][x]) != get_bottom_edge(tile):
            return False
        
    return True
    

In [24]:
def place_tile_on_grid(grid, tile, x, y):
    '''Place tile on grid at x, y in whatever orientation is 
       necessary. Return True if tile can be placed, False
       otherwise.'''
    
    return [oriented_tile for oriented_tile in [tile,
                                                rotate_right(tile),
                                                rotate_right(rotate_right(tile)),
                                                rotate_right(rotate_right(rotate_right(tile))),
                                                flip(tile),
                                                rotate_right(flip(tile)),
                                                rotate_right(rotate_right(flip(tile))),
                                                rotate_right(rotate_right(rotate_right(flip(tile))))]
            if placable_oriented_tile_on_grid(grid, oriented_tile, x, y)]


In [25]:
def find_next_tile(grid, tiles_in, x, y):
    '''Find the tile in the dictionary that fits'''
    
    return [(tile, place_tile_on_grid(grid, tiles_in[tile], x, y))
            for tile in tiles_in
            if place_tile_on_grid(grid, tiles_in[tile], x, y)]
        

In [26]:
def fill_grid(tiles_in):
    
    t_dict=copy.deepcopy(tiles_in)
    
    grid_dim_i=int(math.sqrt(len(t_dict)))
    
    grid_out=[[False for i in range(grid_dim_i)]
              for i in range(grid_dim_i)]

    (tile_id, start_tile)=get_first_tile(t_dict)
    
    grid_out[0][0]=start_tile
    
    t_dict.pop(tile_id)

    for (y, x) in it.islice(it.product(range(grid_dim_i),
                                       range(grid_dim_i)), 1, None):  # drop (0, 0)
        
        (tid, tgrids)=find_next_tile(grid_out, t_dict, x, y)[0]
        t_dict.pop(tid)
        
        grid_out[y][x]=tgrids[0]
       
    return grid_out

        
            

OK, now convert the output to the relevant image:

In [27]:
def peel(grid_in):
    '''return grid without outer layer'''
    return [row[1:-1] for row in grid_in[1:-1]]

In [28]:
def remove_borders(grid_in):
    
    return [[peel(tile) for tile in row]
            for row in grid_in]
        

In [29]:
def append_tiles(tile1_in, tile2_in):
    
    return list(map(operator.add, tile1_in, tile2_in))

In [30]:
def build_image(tile_grid_in):
    '''tile grid is a grid of pre-peeled tiles.
       Return the grid'''
    
    return ft.reduce(operator.add,
                     [ft.reduce(append_tiles, grid_row)
                      for grid_row in tile_grid_in])

Put it together:

In [31]:
def tile_to_strings(grid_in):
    
    return [''.join(row) for row in grid_in]

In [32]:
def find_image(str_in):
    
    return build_image(remove_borders(fill_grid(parse_input(str_in))))


tile_to_strings(find_image(open('data/day20_test').read()))

['.####...#####..#...###..',
 '#####..#..#.#.####..#.#.',
 '.#.#...#.###...#.##.##..',
 '#.#.##.###.#.##.##.#####',
 '..##.###.####..#.####.##',
 '...#.#..##.##...#..#..##',
 '#.##.#..#.#..#..##.#.#..',
 '.###.##.....#...###.#...',
 '#.####.#.#....##.#..#.#.',
 '##...#..#....#..#...####',
 '..#.##...###..#.#####..#',
 '....#.##.#.#####....#...',
 '..##.##.###.....#.##..#.',
 '#...#...###..####....##.',
 '.#.##...#.##.#.#.###...#',
 '#.###.#..####...##..#...',
 '#.###...#.##...#.######.',
 '.###.###.#######..#####.',
 '..##.#..#..#.#######.###',
 '#.#..##.########..#..##.',
 '#.#####..#.#...##..#....',
 '#....##..#.#########..##',
 '#...#.....#..##...###.##',
 '#..###....##.#...##.##.#']

In [33]:
def find_monsters_no_reorientation(grid_in):
    '''don't do the rotations here...'''
    grid_height=len(grid_in)
    grid_width=len(grid_in[0]) 
    
    out=[]
    for (row_i, row_txt) in enumerate(grid_in[:-2]):
        for c in range(len(row_txt)-20):
            if re.match('..................#.', grid_in[row_i][c:]) and \
               re.match('#....##....##....###', grid_in[row_i+1][c:]) and \
               re.match('.#..#..#..#..#..#...', grid_in[row_i+2][c:]):
                
                out.append((row_i, c))
    return out
        
        

In [34]:
def find_monsters(grid_image):

    return [find_monsters_no_reorientation(tile_to_strings(grid))
            for grid in [grid_image,
                         rotate_right(grid_image),
                         rotate_right(rotate_right(grid_image)),
                         rotate_right(rotate_right(rotate_right(grid_image))),
                         flip(grid_image),
                         rotate_right(flip(grid_image)),
                         rotate_right(rotate_right(flip(grid_image))),
                         rotate_right(rotate_right(rotate_right(flip(grid_image))))]]


Let's assume for the moment that the monsters are non-overlapping. Then we can count all the hashes in the grid, and remove the number per monster (15). Also, assume that there's only one orientation containing monsters:

In [35]:
def day20_part2(file_in):

    image=find_image(open(file_in).read())
    
    hashes_num=sum(row.count('#') for row in image)
    
    monster_coords=[c for c in find_monsters(image)
                    if c]

    return hashes_num-15*len(monster_coords[0])



In [36]:
assert day20_part2('data/day20_test')==273

In [37]:
day20_part2('data/day20_input')

2084

Thank heavens for that. That was an almighty slog.

Done!